# Import Librairies

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
from pathlib import Path
import imghdr

# Pre-Process Images to the appropriate types

In [ ]:
data_dir = "./BBTrD/000000001111"
image_extensions = [".png", ".jpg", ".jpeg"]  # add there all your images file extensions

img_type_accepted_by_tf = ["bmp", "gif", "jpeg", "png"]
for filepath in Path(data_dir).rglob("*"):
    if filepath.suffix.lower() in image_extensions:
        img_type = imghdr.what(filepath)
        if img_type is None:
            print(f"{filepath} is not an image")
        elif img_type not in img_type_accepted_by_tf:
            print(f"{filepath} is a {img_type}, not accepted by TensorFlow")

# Load Labels and Images

In [ ]:
def load_labels():
    txt_file_path = "labels.txt"
    labels = []
    with open(txt_file_path, 'r') as f:
        for line in f:
            labels.append(line.strip())
    return labels

In [ ]:
game_states = load_labels()
game_states = sorted(game_states)
game_states

In [ ]:
image_height = 128
image_width = 128
batch_size = 81

def load_data():
    # Load Train Data
    train_images = tf.keras.utils.image_dataset_from_directory("./BBTrD/", batch_size=batch_size, image_size=(image_height, image_width), shuffle=False)

    # Load Test Data
    test_images =tf.keras.utils.image_dataset_from_directory("./BBTeD/", batch_size=batch_size, image_size=(image_height, image_width), shuffle=False)
    return train_images, test_images

In [ ]:
train_images, test_images = load_data()

# View Class Names of loaded data 

In [ ]:
class_names = train_images.class_names
class_names

# Visualize the Loaded Data

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_images.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(game_states[labels[i]])
        plt.axis("off")

# Data Processing

## Resize & Rescale Images

In [ ]:
resize_and_rescale = tf.keras.Sequential([
    layers.Rescaling(1./255)
])

train_images = train_images.map(lambda x, y: (resize_and_rescale(x), y))
test_images = train_images.map(lambda x, y: (resize_and_rescale(x), y))

## Data Augmentation

In [ ]:
data_flip = tf.keras.Sequential([
  layers.RandomFlip("horizontal_and_vertical")
])

data_rotation = tf.keras.Sequential([
  layers.RandomRotation(0.2)
])

flipped_images = train_images.map(lambda x, y: (data_flip(x), y))
rotated_images = train_images.map(lambda x, y: (data_rotation(x), y))

# Define CNN-Model

In [ ]:
num_classes = 81

cnn_model = tf.keras.Sequential([
    # Normalize Data 
    # The RGB channel values are in the [0, 255] range. This is not ideal for a neural network;
    # in general you should seek to make your input values small.
  tf.keras.layers.Rescaling(1./255),
    # First Convulation Layer
  tf.keras.layers.Conv2D(64, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
    # Second Convulation Layer
  tf.keras.layers.Conv2D(64, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
    # Thrid Convulation Layer
  tf.keras.layers.Conv2D(64, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
    
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes)
])

# Add an Optimizer to CNN model

In [ ]:
cnn_model.compile(
  optimizer='adam',
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

# Train Model

In [ ]:
epochNb = 5
cnn_model.fit(train_images,epochs=epochNb)

# Evaluate CNN Model

In [ ]:
cnn_model.evaluate(test_images)

# Predict State

In [ ]:
y_pred = cnn_model.predict(train_images[0])